# OpenAI Fine-tuning API 학습을 위해 데이터셋 만들기

- jsonline 포맷
- 예시
```json
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "What's the capital of France?"}, {"role": "assistant", "content": "Paris, as if everyone doesn't know that already."}]}
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "Who wrote 'Romeo and Juliet'?"}, {"role": "assistant", "content": "Oh, just some guy named William Shakespeare. Ever heard of him?"}]}
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "How far is the Moon from Earth?"}, {"role": "assistant", "content": "Around 384,400 kilometers. Give or take a few, like that really matters."}]}
```

In [1]:
import json
from datasets import load_dataset

In [2]:
dataset = load_dataset("nsmc")

In [3]:
train_valid_ds = dataset['train'].train_test_split(test_size=30)

In [4]:
train_df = train_valid_ds['train'].to_pandas()
valid_df = train_valid_ds['test'].to_pandas()

In [5]:
len(train_df), len(valid_df)

(149970, 30)

In [6]:
# 실습을 위한 학습 시간을 줄이기 위해 샘플을 줄임, 일반적으로 많을 수록 좋음
train_df = train_df.sample(100) 

In [7]:
train_df

,id,document,label
87690,8967787,"영화의 꿈과 감동을 주는 영화의 역사,영화를 위한 영화의 영화",1
120061,9461567,아....공포스럽습니다...목욕바구니가...그리무서울줄이야....이번년도...마지막...,1
36810,8850393,소재만 대박이고 완벽하게 망한 영화 진짜 이 영화도 명품 도자기 가져다가 개밥그릇으...,0
83693,471166,아름답고 감동적인 이야기,1
120099,238991,좋다,1
...,...,...,...
137385,10190703,"스스로가 대견하다. 영화관에서 보지 않고 집에서 본 것이 대견하고, 영화를 끝까지 ...",0
123652,8024536,전문 더빙 불렀으면 내가 10점은 준다 어휴,0
93780,9249676,"어바웃타임, 비긴어게인 보다도 더 감동적이고 여운을 주는 영화.비긴어게인은 성시경 ...",1
8651,8111946,최고다 최고다...무혁아..,1


In [8]:
# JSONL 파일 생성 함수
def create_jsonl(df, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for i, item in df.iterrows():
            doc = item['document']
            label = '긍정' if item['label'] == 1 else '부정'
            json_line = json.dumps({"messages": [{"role": "system", "content": "user의 문장을 긍정 또는 부정으로 응답하라."},
                                                 {"role": "user", "content": doc},
                                                 {"role": "assistant", "content": label}
                                                ],
                                    }
                                  ,ensure_ascii=False)
            f.write(json_line + '\n')

In [9]:
# 훈련 및 검증 데이터셋을 JSONL 파일로 변환
create_jsonl(train_df, 'nsmc_train.jsonl')
create_jsonl(valid_df, 'nsmc_valid.jsonl')